# Code to extract and plot spectra from a single NEON AOP pixel
Stephen Escarzaga 
July 10, 2018
smescarzaga@utep.edu

The goal is to extract a single pixel lfrom a hyperspectral hdf5 file and plot 3 figures, both without removing the bad bands of unusable data and with those bands. We also want to export a csv of the cleaned wavelength and reflectance info for use later.

In [1]:
import h5py

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from copy import copy
import IPython
%matplotlib inline

In [3]:
refl_filename = ("../data/NEON_D17_SJER_DP1_20180401_184649_reflectance.h5")
refl_filename

'../data/NEON_D17_SJER_DP1_20180401_184649_reflectance.h5'

In [4]:
pixel_to_plot = (100,100)
pixel_to_plot

(100, 100)

In [5]:
# read in reflectance hdf5 file
hdf5_file = h5py.File(refl_filename, 'r')
hdf5_file

<HDF5 file "NEON_D17_SJER_DP1_20180401_184649_reflectance.h5" (mode r)>

In [6]:
# Get the file name
file_attrs_string = str(list(hdf5_file.items()))

In [7]:
file_attrs_string_split = file_attrs_string.split("'")
file_attrs_string_split

['[(', 'SJER', ', <HDF5 group "/SJER" (1 members)>)]']

In [8]:
sitename = file_attrs_string_split[1]
sitename

'SJER'

In [9]:
# Exract the reflectance
refl = hdf5_file[sitename]['Reflectance']
refl

<HDF5 group "/SJER/Reflectance" (2 members)>

In [10]:
list(refl.attrs)


['Description',
 'Sensor',
 'PayloadID',
 'Scripts',
 'Acquisition_Time',
 'Band_Window_1_Nanometers',
 'Band_Window_2_Nanometers']

In [12]:
# Get the reflectance data
refldata = refl['Reflectance_Data']
refldata

<HDF5 dataset "Reflectance_Data": shape (216, 699, 426), type "<i2">

In [13]:
# Get the raw values for the reflectance data
reflRaw = refldata.value
reflRaw

array([[[-9999, -9999, -9999, ..., -9999, -9999, -9999],
        [-9999, -9999, -9999, ..., -9999, -9999, -9999],
        [-9999, -9999, -9999, ..., -9999, -9999, -9999],
        ...,
        [-9999, -9999, -9999, ..., -9999, -9999, -9999],
        [-9999, -9999, -9999, ..., -9999, -9999, -9999],
        [-9999, -9999, -9999, ..., -9999, -9999, -9999]],

       [[-9999, -9999, -9999, ..., -9999, -9999, -9999],
        [-9999, -9999, -9999, ..., -9999, -9999, -9999],
        [-9999, -9999, -9999, ..., -9999, -9999, -9999],
        ...,
        [-9999, -9999, -9999, ..., -9999, -9999, -9999],
        [-9999, -9999, -9999, ..., -9999, -9999, -9999],
        [-9999, -9999, -9999, ..., -9999, -9999, -9999]],

       [[-9999, -9999, -9999, ..., -9999, -9999, -9999],
        [-9999, -9999, -9999, ..., -9999, -9999, -9999],
        [-9999, -9999, -9999, ..., -9999, -9999, -9999],
        ...,
        [-9999, -9999, -9999, ..., -9999, -9999, -9999],
        [-9999, -9999, -9999, ..., -9999, -99